In [ ]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import os
import numpy as np
import random

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Flatten, Conv2D, Dense, Dropout, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
% cd /content/drive/MyDrive/Colab Notebooks/

In [ ]:
base_dir = ""
train_path = os.path.join(base_dir, 'train/')
test_path = os.path.join(base_dir, 'test/')
val_path = os.path.join(base_dir, 'validation/')

In [ ]:
print("Number of train images in each Category")
print("==========================================")
print("Number of directories: ", len(os.listdir(train_path)))
for dir in os.listdir(train_path):
  print(dir, len(os.listdir(train_path+dir)))

print("Number of test images in each Category")
print("==========================================")

for dir in os.listdir(test_path):
  print(dir, len(os.listdir(test_path+dir)))


print("Number of validation images in each Category")
print("==========================================")

for dir in os.listdir(val_path):
  print(dir, len(os.listdir(val_path+dir)))

In [ ]:
random.sample(os.listdir(train_path+''), 1)

In [ ]:
from tensorflow.python.ops.gen_logging_ops import image_summary
k=1
plt.figure(figsize=(16,14))
for dir in os.listdir(train_path):
  random_img = random.sample(os.listdir(train_path+dir),1)
  img = load_img(train_path+dir+'/'+random_img[0], target_size=(224,224))

  i = img_to_array(img)/255.
  img_arr = np.array(i)
  plt.subplot(3,5,k)
  plt.imshow(img_arr)
  plt.title("org: {}".format(dir))
  plt.axis('off')
  k+=1

plt.show()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255.,
                                   rotation_range=30,
                                   height_shift_range=0.1,
                                   width_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1/255.)
val_datagen = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=(150,150),
                                                    class_mode='categorical',
                                                    batch_size=100)

test_generator = test_datagen.flow_from_directory(test_path,
                                                  target_size=(150,150),
                                                  class_mode='categorical',
                                                  batch_size=100)
val_generator = val_datagen.flow_from_directory(val_path, 
                                                target_size=(150,150),
                                                class_mode='categorical',
                                                batch_size=100)

In [ ]:
test_generator = test_datagen.flow_from_directory(test_path,
                                                  target_size=(150,150),
                                                  class_mode='categorical',
                                                  batch_size=100,
                                                  shuffle=False)

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', input_shape=(150,150,3)))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(15, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
mc = ModelCheckpoint('', monitor='val_loss', mode='min', save_best_only=True)

es = EarlyStopping(monitor='accuracy', min_delta=0.01, patience=3, mode='max')

In [ ]:
hist = model.fit(train_generator, epochs=30, validation_data=val_generator, callbacks=[mc,es])

In [ ]:
valloss, valaccuracy = model.evaluate(val_generator)

testloss, testaccuracy = model.evaluate(test_generator)


print("\nValidation accuracy: {}, validation loss: {}".format(valaccuracy, valloss))
print("\nTest accuracy: {}, Test loss: {}".format(testaccuracy, testloss))

In [ ]:
acc = hist.history['accuracy']
loss = hist.history['loss']
val_acc = hist.history['val_accuracy']
val_loss = hist.history['val_loss']
epochs = range(1, len(acc)+1)

plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.plot(epochs, acc, 'ro--', label='Accuracy')
plt.plot(epochs, val_acc, 'g^:', label='Val_Accuracy')
plt.title("Accuracy vs Epochs")
plt.xlabel("Epochs", fontsize=12)
plt.ylabel("Accuracy", fontsize=12)
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, loss, 'ro--', label='Loss')
plt.plot(epochs, val_loss, 'g^:', label='Val_Loss')
plt.title("Loss vs Epochs")
plt.xlabel("Epochs", fontsize=12)
plt.ylabel("Loss", fontsize=12)
plt.legend()

plt.show()

## Loading the saved model at best val_loss

In [ ]:
model1 = load_model('')

In [ ]:
valloss, valaccuracy = model1.evaluate(val_generator)

testloss, testaccuracy = model1.evaluate(test_generator)

print("\nValidation accuracy: {}, validation loss: {}".format(valaccuracy, valloss))
print("\nTest accuracy: {}, Test loss: {}".format(testaccuracy, testloss))

In [ ]:
prediction = model1.predict(test_generator)

In [ ]:
prediction[3], np.argmax(prediction[2])

In [ ]:
y_pred = [np.argmax(i) for i in prediction]
#y_pred = np.argmax(prediction, axis=1)

In [ ]:
y_pred[0:5]

## Predict Random Images

In [ ]:
img_arr.reshape(-1,150,150,3).shape

In [ ]:
class_dict = test_generator.class_indices
class_list = []
for item, num in class_dict.items():
  class_list.append(item)

class_list

In [ ]:
pred = model1.predict(img_arr.reshape(-1,150,150,3))
class_list[np.argmax(pred)]

In [ ]:
img = load_img('', target_size=(150,150))

i = img_to_array(img)/255.
img_arr = np.array(i)
plt.imshow(img_arr)
plt.axis('off')
pred = model1.predict(img_arr.reshape(-1, 150, 150, 3))
plt.title("pred: {}".format(class_list[np.argmax(pred)]) )
plt.show()

In [ ]:
k=1
plt.figure(figsize=(16,24))

for dir in os.listdir(test_path):
  for i in range(2):
    random_img = random.sample(os.listdir(test_path+dir),1)
    img = load_img(test_path+dir+'/'+random_img[0], target_size=(150,150))

    i = img_to_array(img)/255.
    plt.subplot(6,5,k)
    img_arr = np.array(i)
    plt.imshow(img_arr)
    pred = model1.predict(img_arr.reshape(-1,150,150,3))
    plt.title("Org: {}\nPred: {}".format(dir, class_list[np.argmax(pred)] ))
    plt.axis('off')
    k+=1

plt.show()

## Getting Performance Scores

In [ ]:
y_true = test_generator.classes

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)
cm

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(cm, annot = True, fmt='d')
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=class_list) )

# Build a model using Transfer Learning

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

In [ ]:
base_model = MobileNetV2(input_shape= (150,150,3), include_top=False, weights='imagenet')

In [ ]:
for layers in base_model.layers:
  layers.trainable=False

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D
model_mnet = Sequential()
model_mnet.add(base_model)
model_mnet.add(GlobalAveragePooling2D())
model_mnet.add(Dense(1024, activation='relu'))
model_mnet.add(Dense(15, activation='softmax'))

model_mnet.summary()

In [ ]:
model_mnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = datagen.flow_from_directory(train_path,
                                              target_size=(150,150),
                                              class_mode='categorical',
                                              batch_size=100)

val_generator = datagen.flow_from_directory(val_path,
                                            target_size=(150,150),
                                            class_mode='categorical',
                                            batch_size=100)

In [ ]:
hist_mnet = model_mnet.fit(train_generator, epochs=30, validation_data=val_generator, callbacks=[es])

In [ ]:
save_model(model_mnet, 'classifier_mnet.h5')

In [ ]:
test_generator = datagen.flow_from_directory(test_path,
                                             target_size=(150,150),
                                             class_mode='categorical',
                                             batch_size=100,
                                             shuffle=False)

In [ ]:
valloss, valaccuracy = model_mnet.evaluate(val_generator)

testloss, testaccuracy = model_mnet.evaluate(test_generator)


print("\nValidation accuracy: {}, validation loss: {}".format(valaccuracy, valloss))
print("\nTest accuracy: {}, Test loss: {}".format(testaccuracy, testloss))

In [ ]:
acc = hist_mnet.history['accuracy']
loss = hist_mnet.history['loss']
val_acc = hist_mnet.history['val_accuracy']
val_loss = hist_mnet.history['val_loss']
epochs = range(1, len(acc)+1)

plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.plot(epochs, acc, 'ro--', label='Accuracy')
plt.plot(epochs, val_acc, 'g^:', label='Val_Accuracy')
plt.title("Accuracy vs Epochs")
plt.xlabel("Epochs", fontsize=12)
plt.ylabel("Accuracy", fontsize=12)
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, loss, 'ro--', label='Loss')
plt.plot(epochs, val_loss, 'g^:', label='Val_Loss')
plt.title("Loss vs Epochs")
plt.xlabel("Epochs", fontsize=12)
plt.ylabel("Loss", fontsize=12)
plt.legend()

plt.show()

# Getting Performance Score

In [ ]:
prediction_mnet = model_mnet.predict(test_generator)

In [ ]:
y_pred_mnet = np.argmax(prediction_mnet, axis=1)

In [ ]:
y_test_mnet = test_generator.classes
y_test_mnet[:6]

In [ ]:
y_pred_mnet[:6]

In [ ]:
cm_mnet = confusion_matrix(y_test_mnet, y_pred_mnet)
cm_mnet

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(cm_mnet, annot = True, fmt='d')
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.show()

In [ ]:
print(classification_report(y_test_mnet, y_pred_mnet, target_names=class_list) )

## Predicting some random images

In [ ]:
k=1
plt.figure(figsize=(16,24))

for dir in os.listdir(test_path):
  for i in range(2):
    random_img = random.sample(os.listdir(test_path+dir),1)
    img = load_img(test_path+dir+'/'+random_img[0], target_size=(150,150))

    i = img_to_array(img)/255.
    plt.subplot(6,5,k)
    img_arr = np.array(i)
    plt.imshow(img_arr)
    pred = model_mnet.predict(img_arr.reshape(-1,150,150,3))
    plt.title("Org: {}\nPred: {}".format(dir, class_list[np.argmax(pred)] ))
    plt.axis('off')
    k+=1

plt.show()